In [1]:
import warnings; warnings.filterwarnings('ignore')
import numpy as np
import oxyba as ox; from importlib import reload; reload(ox);
from time import perf_counter, process_time

### Load Demo Dataset

In [2]:
from sklearn.datasets import load_boston
tmp = load_boston()
num_obs = len(tmp.target);
y = tmp.target
X = np.c_[ np.ones(shape=(num_obs,1)), tmp.data[:,[5,12]] ];

### OLS with Inverse from Singular Value Decomposition (SVD)
The regressions coefficients 

$$
\hat{\beta} = (X^T X)^{-1} - (X^T y)
$$

will be estimated by compute the inverse matrix $(X^T X)^{-1}$ with Singular Value Decomposition (SVD).

SVD decompose a matrix $A$ into 

$$
A = U S V^T
$$

with $U U^T = I$ and $V V^T = I$ and $S$ a diagonal matrix.

In [3]:
from numpy import dot, diag
from numpy.linalg import svd
u,s,v = svd(dot(X.T, X));

In [4]:
print('U\n',u)
print('U*U^T\n', dot(u,u.T).round(2))

U
 [[-0.05849311 -0.12435103 -0.99051268]
 [-0.35644426 -0.9242065   0.13707605]
 [-0.9324838   0.36108057  0.0097355 ]]
U*U^T
 [[ 1.  0. -0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]


In [5]:
print('V\n',v)
print('V*V^T\n',dot(v,v.T).round(2))

V
 [[-0.05849311 -0.35644426 -0.9324838 ]
 [-0.12435103 -0.9242065   0.36108057]
 [-0.99051268  0.13707605  0.0097355 ]]
V*V^T
 [[ 1. -0.  0.]
 [-0.  1.  0.]
 [ 0.  0.  1.]]


In [6]:
print('s\n',s)
print('S\n',diag(s))

s
 [1.21950783e+05 5.54978203e+03 2.99151160e+00]
S
 [[1.21950783e+05 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 5.54978203e+03 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.99151160e+00]]


The inverse is

$$
A^{-1} = V S^{-1} U^T
$$

In [7]:
ainv = dot(dot(v, diag(1.0/s)), u.T); # 1/s = s**-1
ainv

array([[ 3.08760632e-01, -4.26684318e-02, -3.05739484e-03],
       [-1.19535807e-01,  1.66995849e-02,  1.11591123e-03],
       [-3.22609506e-03,  4.26164541e-04,  4.81752593e-05]])

Just to crush all hopes - It does not seem to work.
Compared to Numpy's `inv` the SVD based inverse seems similar but is off.

In [8]:
np.linalg.inv(dot(X.T, X))

array([[ 3.27969244e-01, -4.53660623e-02, -3.23114204e-03],
       [-4.53660623e-02,  6.43600293e-03,  3.88691406e-04],
       [-3.23114204e-03,  3.88691406e-04,  6.23057559e-05]])

### Test Implementations
The first implementation applies Numpy's [svd](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.linalg.svd.html) what is based on LAPACK's [_gesdd](http://www.netlib.org/lapack/explore-html/d1/d7e/group__double_g_esing_gad8e0f1c83a78d3d4858eaaa88a1c5ab1.html#gad8e0f1c83a78d3d4858eaaa88a1c5ab1).
`func1_pretty` is the more readable but slower version of `func1_faster`.

In [9]:
def func1_pretty(y,X):
    from numpy import dot, diag
    from numpy.linalg import svd
    u,s,v = svd(dot(X.T, X));
    x2inv = dot(v,dot(diag(1.0/s),u.T)); # 1/s = s**-1
    return dot(x2inv, dot(X.T,y));

In [10]:
def func1_faster(y,X):
    import numpy as np
    u,s,v = np.linalg.svd(np.dot(X.T, X));
    return np.dot(np.dot(v,np.dot(np.diag(s**-1),u.T)), np.dot(X.T,y)); 

The second implementation is a hack in case $A A^T$ is an ill-conditioned or resp. singular matrix. 
The diagnonal elements $S$ are required to greater than a certain threshold `tol` to compute $1/s_i$ or are set to zero.

In [11]:
def func2_pretty(y, X, tol=1e-16):
    from numpy import dot, diag
    from numpy.linalg import svd
    u,s,v = svd(dot(X.T, X));
    adj = [1.0/sigma  if sigma>tol else 0.0 for sigma in s];
    x2inv = dot(v,dot(diag(adj),u.T)); # 1/s = s**-1
    return dot(x2inv, dot(X.T,y)) 

In [12]:
def func2_faster(y, X, tol=1e-16):
    import numpy as np
    u,s,v = np.linalg.svd(np.dot(X.T, X));
    return dot(np.dot(v,np.dot(np.diag([1.0/sigma  if sigma>tol else 0.0 for sigma in s]),u.T)), np.dot(X.T,y));

The third implementation applies Numpy's [lstsq](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.lstsq.html) is based on LAPACK's [_gelsd](http://www.netlib.org/lapack/explore-html/d7/d3b/group__double_g_esolve_ga94bd4a63a6dacf523e25ff617719f752.html) what applies SVD to solve problems of type

$$
\min_x || b - A x ||_2
$$

For a Linear Regression problem the variables are $A=X^T X$, $x=\beta$ and $b=X^T y$.



In [13]:
def func3_pretty(y,X):
    from numpy.linalg import lstsq
    beta,_,_,_ = lstsq(b=y, a=X, rcond=-1)
    return beta

In [14]:
def func3_faster(y,X):
    import numpy as np
    beta,_,_,_ = np.linalg.lstsq(b=y, a=X, rcond=-1)
    return beta

### Benchmarking

In [15]:
trials = 50000
funcnames = [func1_pretty, func1_faster, 
             func2_pretty, func2_faster, 
             func3_pretty, func3_faster, 
             ox.linreg_ols_svd, 
             ox.linreg_ols_lu]

In [16]:
for func in funcnames:
    beta = func(y,X);
    print(func.__name__, beta);

func1_pretty [-0.13256476  5.28717637  0.49253069]
func1_faster [-0.13256476  5.28717637  0.49253069]
func2_pretty [-0.13256476  5.28717637  0.49253069]
func2_faster [-0.13256476  5.28717637  0.49253069]
func3_pretty [-1.35827281  5.09478798 -0.64235833]
func3_faster [-1.35827281  5.09478798 -0.64235833]
linreg_ols_svd [-1.35827281  5.09478798 -0.64235833]
linreg_ols_lu [-1.35827281  5.09478798 -0.64235833]


That does not look good. 
`func1_...` and `func2_...` differ from the already tested solution of [linreg_ols_lu](http://oxyba.de/docs/linreg_ols_lu/).

In [17]:
print('{0:6s} {1:6s} {2:s}'.format('Clock', 'CPU', 'function name'))
for func in funcnames:
    sh,sc = perf_counter(), process_time();
    for i in range(trials):
        beta = func(y,X); 
        if beta is None: print('error solving')
        beta = None;
    eh,ec = perf_counter(), process_time()
    print('{0:.4f} {1:.4f} {2:s}'.format(eh-sh, ec-sc, func.__name__))

Clock  CPU    function name
4.2338 4.0752 func1_pretty
3.7378 3.7075 func1_faster
4.4396 4.4287 func2_pretty
4.2982 4.2780 func2_faster
7.8053 6.2520 func3_pretty
6.0752 5.8702 func3_faster
10.3562 8.0228 linreg_ols_svd
2.1758 2.1564 linreg_ols_lu


The `func3_...` implementation based on Numpy's `lstsq` is the only feasible option for the `linreg_ols_svd` wrapper because the direct implementation based on Numpy's `svd` does not work correctly (There must be something wrong with my code ...). 

`func3_pretty` are not much different `func3_faster` what indicates that the computation itself takes much longer than overhead due to things like importing modules. 

Linear Regression based on LU Decomposition (`linreg_ols_lu`) is much faster than based on Singular Value Decomposition (`linreg_ols_svd`). 